In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
from IPython.display import display
from scipy.interpolate import interp1d
import seaborn as sns
import qgrid
qgrid.set_grid_option('forceFitColumns', False)
from tqdm import tqdm

In [ ]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
from pathlib import Path
data_dir = Path("../csv_data/")

data = {}
for csv_file in data_dir.glob("*.csv"):
    if csv_file.stem in ["schmelzen", "FormateCC4"]:continue
        
    this_csv = pd.read_csv(csv_file, delimiter=";")
    
    this_csv['DATE_TIME'] = pd.to_datetime(this_csv['TIME'],unit='s').dt.tz_localize("UTC")
    this_csv = this_csv.set_index('DATE_TIME', drop=False).sort_index()
    
    data[csv_file.stem] = this_csv.sort_values("TIME")
    #display( data[csv_file.stem].tail())
    
data['TundishTemperaturInC'] = data['TundishTemperaturInC'][(data['TundishTemperaturInC']['TundishTemperaturInC'] < 1600) & (data['TundishTemperaturInC']['TundishTemperaturInC'] > 1400)]  
    

### Import and further Preprocessing of schmelzen Dataframe

In [ ]:
df_schmelzen = pd.read_pickle("../final_data/schmelzen.pkl")

In [ ]:
cols_schmelzen = ['ChargenNr',
 'ChargenNrErsteSchmInSeq',
 'GiessBeginnSchmelze',
 'GiessBeginn_DateTime',
 'GiessEndeSchmelze',
 'GiessEnde_DateTime',
 'EndeSchmelze',
 'Ende_DateTime',
 'NrSchmelzeInSequenz',
 'LiquidusTempInC',
 'SolidusTempInC',
 'UeberhitzungMittelInK',
 'ZielTempTreiberInC',
 'Str1SollGiessGeschwInMproMin',
 'Str2SollGiessGeschwInMproMin',
 'Str1Format',
 'Str2Format',
 'C-Aequiv01',
 'C-Aequiv02',
 'C-AequivP']

#### Add 90 minutes to the end of the last melting process of a sequence

In [ ]:
df_schmelzen['EndeSchmelze'] = df_schmelzen['GiessEndeSchmelze']

df_schmelzen['ChargenNrErsteSchmInSeq_shifted'] = df_schmelzen['ChargenNrErsteSchmInSeq'].shift(-1)
df_schmelzen = df_schmelzen.fillna(0)

def add_time_to_end(row):               
        if row['ChargenNrErsteSchmInSeq'] !=  row['ChargenNrErsteSchmInSeq_shifted']:
            row['EndeSchmelze'] = row['EndeSchmelze'] + 90*60   #add 90 min to end of last 'GiessEnde' 
        return row

df_schmelzen = df_schmelzen.apply(add_time_to_end, axis=1)

df_schmelzen['Ende_DateTime'] = pd.to_datetime(df_schmelzen['EndeSchmelze'],unit='s').dt.tz_localize("UTC")

df_schmelzen = df_schmelzen.drop("ChargenNrErsteSchmInSeq_shifted", axis=1)

df_schmelzen = df_schmelzen[cols_schmelzen]

## Combine df_schmelzen and Laenge Sequenz with 90min added to 'GiessEnde'

In [ ]:
merged = []

for _, row in data['Str1GiessLaengeSequenzInM'].iterrows():
    schmelze = df_schmelzen[(df_schmelzen["GiessBeginn_DateTime"] < row["DATE_TIME"]) & (df_schmelzen["Ende_DateTime"] >= row["DATE_TIME"])]
    if len(schmelze) != 1:
        continue
    merged.append(pd.concat([row.to_frame().transpose().reset_index(drop=True), schmelze.reset_index(drop=True)], axis=1))

df_str1_extended = pd.concat(merged)
df_str1_extended = df_str1_extended.reset_index(drop=True)

## Combine df_schmelzen and Laenge Sequenz

In [ ]:
merged = []

for _, row in data['Str1GiessLaengeSequenzInM'].iterrows():
    schmelze = df_schmelzen[(df_schmelzen["GiessBeginn_DateTime"] < row["DATE_TIME"]) & (df_schmelzen["GiessEnde_DateTime"] >= row["DATE_TIME"])]
    if len(schmelze) != 1:
        continue
    merged.append(pd.concat([row.to_frame().transpose().reset_index(drop=True), schmelze.reset_index(drop=True)], axis=1))

df_merged_str1 = pd.concat(merged)
df_merged_str1 = df_merged_str1.reset_index(drop=True)

## Combine df_schmelzen and Laenge Schmelze nothing added to 'GiessEnde'

In [ ]:
merged = []

for _, row in data['Str1GiessLaengeSchmelzeInM'].iterrows():
    schmelze = df_schmelzen[(df_schmelzen["GiessBeginn_DateTime"] < row["DATE_TIME"]) & (df_schmelzen["GiessEnde_DateTime"] >= row["DATE_TIME"])]
    if len(schmelze) != 1:
        continue
    merged.append(pd.concat([row.to_frame().transpose().reset_index(drop=True), schmelze.reset_index(drop=True)], axis=1))

df_str1_schmelzen = pd.concat(merged)
df_str1_schmelzen = df_str1_schmelzen.reset_index(drop=True)

## Create interpolated Data

### Interpolated Sequence data

In [ ]:
df_seq_str1 = data['Str1GiessLaengeSequenzInM'].copy()

df_seq_str1 = df_seq_str1[df_seq_str1['DATE_TIME'] >= '2019-05-15 11:30:00+00:00' ]

In [ ]:
rng = pd.date_range(start='2019-05-15 11:31:00+00:00', end='2019-08-30 23:59:00+00:00', freq='30s')
df_times = pd.DataFrame({'DateTime': rng})
df_times['ts'] = df_times.DateTime.values.astype(np.int64) // 10**9    #np arry by values, then 10**9 for ns

time = df_seq_str1['TIME'].to_numpy() 
length = df_seq_str1['Str1GiessLaengeSequenzInM'].to_numpy()

from scipy.interpolate import interp1d
interp_s = interp1d(time, length, kind='linear')

new_time = df_times['ts'].to_numpy()

inter_array = interp_s(new_time)

str1_inter = pd.DataFrame({'DATE_TIME':new_time , 'Str1GiessLaengeSequenzInM': inter_array, 'TIME': new_time})
str1_inter['DATE_TIME'] = pd.to_datetime(str1_inter['DATE_TIME'],unit='s').dt.tz_localize("UTC")
str1_inter.head()

## Combine df_schmelzen and str1_inter with 90min added to 'GiessEnde'

In [ ]:
merged = []

for _, row in str1_inter.iterrows():
    schmelze = df_schmelzen[(df_schmelzen["GiessBeginn_DateTime"] < row["DATE_TIME"]) & (df_schmelzen["Ende_DateTime"] >= row["DATE_TIME"])]
    if len(schmelze) != 1:
        continue
    merged.append(pd.concat([row.to_frame().transpose().reset_index(drop=True), schmelze.reset_index(drop=True)], axis=1))

str1_inter_extended = pd.concat(merged)
str1_inter_extended = str1_inter_extended.reset_index(drop=True)

## Combine df_schmelzen and str1_inter (nothing added)

In [ ]:
merged = []

for _, row in str1_inter.iterrows():
    schmelze = df_schmelzen[(df_schmelzen["GiessBeginn_DateTime"] < row["DATE_TIME"]) & (df_schmelzen["GiessEnde_DateTime"] >= row["DATE_TIME"])]
    if len(schmelze) != 1:
        continue
    merged.append(pd.concat([row.to_frame().transpose().reset_index(drop=True), schmelze.reset_index(drop=True)], axis=1))

df_str1_inter = pd.concat(merged)
df_str1_inter = df_str1_inter.reset_index(drop=True)

### Outliar detection: Get Outliars with Schmelzen data

In [ ]:
str1_schmelzen_grouped = str1_schmelzen.groupby(str1_schmelzen['ChargenNr'])
df_schmelzen_max = str1_schmelzen_grouped['Str1GiessLaengeSchmelzeInM'].max().to_frame()
df_schmelzen_max = df_schmelzen_max.rename(columns={'Str1GiessLaengeSchmelzeInM': 'Str1GiessLaengeSchmelzeInM_max'})
df_schmelzen_max = df_schmelzen_max.reset_index(drop=False)

In [ ]:
df_schmelzen_max.min()

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(1,1,1)
ax1.plot('ChargenNr', 'Str1GiessLaengeSchmelzeInM_max', data = df_schmelzen_max)

In [ ]:
df_schmelzen_max.tail()

In [ ]:
schmelzen_drop = df_schmelzen_max[df_schmelzen_max['Str1GiessLaengeSchmelzeInM_max']<10]
schmelzen_drop = schmelzen_drop['ChargenNr'].to_numpy()
schmelzen_drop

In [ ]:
# str1_schmelzen[str1_schmelzen['ChargenNr']==473127]

### identify the sequences of schmelzen_drop

In [ ]:
schmelzen = pd.read_pickle("../final_data/df_schmelzen.pkl")

schmelzen = schmelzen.set_index('ChargenNr',drop=True)

seq_drop = schmelzen.loc[schmelzen_drop]

seq_drop = seq_drop['ChargenNrErsteSchmInSeq']
seq_drop = seq_drop.drop_duplicates(keep='first')
seq_drop = seq_drop.reset_index(drop=True)

### Look at min charge

In [ ]:
str1_schmelzen_grouped = str1_schmelzen.groupby(str1_schmelzen['ChargenNr'])
df_schmelzen_min = str1_schmelzen_grouped['Str1GiessLaengeSchmelzeInM'].min().to_frame()
df_schmelzen_min = df_schmelzen_min.rename(columns={'Str1GiessLaengeSchmelzeInM': 'Str1GiessLaengeSchmelzeInM_min'})
df_schmelzen_min = df_schmelzen_min.reset_index(drop=False)

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(1,1,1)
ax1.plot('ChargenNr', 'Str1GiessLaengeSchmelzeInM_min', data = df_schmelzen_min)

## Drop Sequences - create df_strang_1

In [ ]:
df_interv = df_interv.set_index(df_interv['ChargenNrErsteSchmInSeq'],drop=True)
df_interv = df_interv.drop(seq_drop, axis=0)
df_interv = df_interv.reset_index(drop=True)


## Create df_chargen_nr

### calculate min and max for each charge

In [ ]:
df_interv_grouped = df_interv.groupby(df_interv['ChargenNr'])

df_interv_grouped_max = df_interv_grouped['Str1GiessLaengeSequenzInM'].max().to_frame()
df_interv_grouped_min = df_interv_grouped['Str1GiessLaengeSequenzInM'].min().to_frame()

df_interv_grouped_max = df_interv_grouped_max.rename(columns={'Str1GiessLaengeSequenzInM': 'Str1GiessLaengeSequenzInM_max'})

df_interv_grouped_min = df_interv_grouped_min.rename(columns={'Str1GiessLaengeSequenzInM': 'Str1GiessLaengeSequenzInM_min'})

chargen_nr = pd.merge(df_interv_grouped_min,df_interv_grouped_max,on=['ChargenNr'],
                                            how='outer')

chargen_nr = chargen_nr.reset_index(drop=False)

#### set min = 0:

In [ ]:
def set_value_to_zero (row):
    if row['Str1GiessLaengeSequenzInM_min'] < 1:
        row['Str1GiessLaengeSequenzInM_min'] = 0
    return row


chargen_zero = chargen_nr.apply(set_value_to_zero, axis=1)

### Merge df_schmelzen and chargen_zero to get seq_id

In [ ]:
df_chargen = pd.merge(chargen_zero,df_schmelzen[['ChargenNr','ChargenNrErsteSchmInSeq']],on=['ChargenNr'],
                                            how='inner')
df_chargen = pd.merge(df_chargen,df_schmelzen[['ChargenNr','GiessBeginn_DateTime']],on=['ChargenNr'],
                                            how='inner')  #to sort the values, sometimes the chargen_nr is not ascending order

df_chargen =df_chargen.set_index('GiessBeginn_DateTime', drop=False).sort_index()

df_chargen = df_chargen.reset_index(drop=True)
df_chargen = df_chargen.drop('GiessBeginn_DateTime', axis=1)

### chargen_old

In [ ]:
df_chargen_old = df_chargen.copy()

#### set 'Str1GiessLaengeSequenzInM_min' of i equal to 'Str1GiessLaengeSequenzInM_max' of i-1

df_chargen['Str1GiessLaengeSequenzInM_max_prev'] = df_chargen['Str1GiessLaengeSequenzInM_max'].shift(1)
df_chargen['ChargenNrErsteSchmInSeq_prev'] = df_chargen['ChargenNrErsteSchmInSeq'].shift(1)

def set_min (row):
    if row['ChargenNrErsteSchmInSeq_prev'] == row['ChargenNrErsteSchmInSeq']:
        row['Str1GiessLaengeSequenzInM_min'] = row['Str1GiessLaengeSequenzInM_max_prev']
    return row

df_chargen_nr = df_chargen.apply(set_min, axis=1)
df_chargen_nr = df_chargen_nr.drop(['Str1GiessLaengeSequenzInM_max_prev','ChargenNrErsteSchmInSeq_prev' ], axis=1)

In [ ]:
df_chargen_nr.head()

## Remove time dependence by assigning the specific cooling zone in which each section is 

In [ ]:
def get_charNr(time, seq_nr):
        return df_schmelzen.loc[(((df_schmelzen['GiessBeginnSchmelze'] <= time) & (df_schmelzen['EndeSchmelze'] >= time)) & (df_schmelzen['ChargenNrErsteSchmInSeq'] == seq_nr)), ['ChargenNr']].to_numpy()[0][0]

In [ ]:
df_schmelzen = pd.read_pickle("../final_data/schmelzen.pkl")

def get_charNr(time, seq_nr):
    try:
        return df_schmelzen.loc[(((df_schmelzen['GiessBeginnSchmelze'] <= time) & (df_schmelzen['EndeSchmelze'] >= time)) & (df_schmelzen['ChargenNrErsteSchmInSeq'] == seq_nr)), ['ChargenNr']].to_numpy()[0][0]
    except IndexError:
        return df_schmelzen.loc[(((df_schmelzen['GiessBeginnSchmelze'] <= np.round(time.item(),-1)) & (df_schmelzen['EndeSchmelze'] >=  np.round(time.item(),-1))) & (df_schmelzen['ChargenNrErsteSchmInSeq'] == seq_nr)), ['ChargenNr']].to_numpy()[0][0]

def charNr(row):
    time =row['z1_begin_time']
    row['ChargenNr'] = get_charNr(time, row['SequenzNr'])
    return row    

df_chargen_nr = pd.read_pickle("../final_data/chargen_nr.pkl")

# Zonen Start und Ende
zonen = {'z1':[0.62,2.17],'z2a':[2.17,3.52],'z2b':[3.52,5.44],'z3a':[5.44,7.18],'z3b':[7.18,9.1],'z4':[9.1,11.02],'z5':[11.02,14.42],'zpy':[15.42,15.42]}

# seq_max = pd.read_pickle("/home/di40438/bachelorarbeit/data/merged_1_all.pkl") #'sequenz_laenge' nur bis zum Giessende 
seq_drop = pd.read_pickle("../final_data/seq_drop.pkl")

seq_max = pd.read_pickle("../final_data/df_str1_inter.pkl") #'sequenz_laenge' nur bis zum Giessende 

seq_max = seq_max.set_index(seq_max['ChargenNrErsteSchmInSeq'],drop=True)

seq_max = seq_max.drop(seq_drop, axis=0)  #drop the chargen from the outliar detection (sequences were already dropped)

seq_max = seq_max.reset_index(drop=True)

str1_seq = pd.read_pickle("../final_data/df_strang_1.pkl") #sequenz_laenge auch noch später als Giessende

sequence_nr = pd.read_pickle("../final_data/df_strang_1.pkl")
sequence_nr = sequence_nr['ChargenNrErsteSchmInSeq']
sequence_nr = sequence_nr.drop_duplicates(keep='first')
sequence_nr = sequence_nr.reset_index(drop=True)

In [ ]:
str1_data = {}
for sequence in tqdm(sequence_nr):   # für alle Sequenzen
    
    seq_max_length = seq_max[seq_max['ChargenNrErsteSchmInSeq']==sequence]
    seq_max_length = seq_max_length.reset_index(drop=True)

    seq = str1_seq[str1_seq['ChargenNrErsteSchmInSeq']==sequence]
    seq = seq.reset_index(drop=True)

    gil = seq['Str1GiessLaengeSequenzInM']  # Länge der Strecke, die das Förderband zurückgelegt hat
    max_gil = np.max(gil)   # maximale Länge der Strecke, die das Förderband zurückgelegt hat
    max_meter = np.max(seq_max_length['Str1GiessLaengeSequenzInM']) # tatsächliche maximale Länge der Schmelze 

    arr = np.arange(0, max_gil, 0.01)
    df_length = pd.DataFrame({'length': arr})

    time = seq['TIME'].to_numpy() 
    length = seq['Str1GiessLaengeSequenzInM'].to_numpy()

    from scipy.interpolate import interp1d
    interp = interp1d(length, time, kind='linear')  #x-Achse: Sequenzlänge, y-Achse: Zeitpunkte

    gilz = []
    cnt  = 0
    section_length = 0.33        #Länge der Abschnitte in Meter
    for section in range(int(np.max((max_meter+section_length)/section_length))):  #für jeden 0.33m-Abschnitt der Sequenz  
        gilz.append({})                                                            # (der ite Abschnitt)
        for z in zonen:        #jede Zone --> Verfolgen jedes Abschnitten durch jede Zone
            mnt = 0
            mxt = 0

            mnt = interp(zonen[z][0]+(section*section_length))   #die Länge des Stranges bei jedem Abschnitt
            mxt = interp(zonen[z][1]+((section+1)*section_length))
            dt  = mxt-mnt

            gilz[-1][z] = [mnt,mxt,dt]

    seq_df = pd.DataFrame()
    for section in range(len(gilz)):
        z_df = pd.DataFrame()
        for z in zonen:
            section_z_df = pd.DataFrame(gilz[section][z], index=[z+'_begin_time',z+'_end_time',z+'_delta_time']).transpose()
            z_df = pd.concat([z_df, section_z_df], axis=1)

        seq_df = pd.concat([seq_df,z_df], axis=0)

        seq_df['SequenzNr'] = seq['ChargenNrErsteSchmInSeq']
        seq_df = seq_df.apply(charNr, axis=1)
        seq_df = seq_df.reindex(sorted(seq_df.columns), axis=1)
        seq_df = seq_df.reset_index(drop=True)

        str1_data[sequence] = seq_df.copy()

## combine all dataframes

In [ ]:
str1_df = pd.DataFrame()
for seq_nr in list(str1_data.keys()):
     str1_df = pd.concat([str1_df,str1_data[seq_nr]], axis=0)

### Load the Combined Data

In [ ]:
str1_df = pd.read_pickle("../final_data/str1_inter_df_0.33_2.pkl") 

## Add water of each zone

### Interpolation functions

In [ ]:
discr = list(data.keys())

In [ ]:
unwanted = {'TundishTemperaturInC',
 'Str1TempMittelLsInC',
 'Str1TempMittelFsInC',
 'Str2TempMittelLsInC',
 'Str2TempMittelFsInC','Str1GiessLaengeSchmelzeInM',
 'Str2GiessLaengeSchmelzeInM','Str1GiessLaengeSequenzInM',
 'Str2GiessLaengeSequenzInM','Str2AusfLaengeSchmelzeInM',
 'Str1AusfLaengeSchmelzeInM',
 'Str1GiessGeschwInMproMin',
 'Str2GiessGeschwInMproMin','Str2WasserZ1FsInLproMin',
 'Str2WasserZ1LsInLproMin',
 'Str2WasserZ2bFsInLproMin',
 'Str2WasserZ4FsInLproMin',
 'Str2WasserZ4LsInLproMin',
 'Str2WasserZ3bLsInLproMin',
 'Str2WasserZ3bFsInLproMin',
 'Str2WasserZ1DiefflenInLproMin',
 'Str2WasserZ2bLsInLproMin',
 'Str2WasserZ1DillingenInLproMin',
 'Str2WasserZ2aFsInLproMin',
 'Str2WasserZ2aLsInLproMin',
 'Str2WasserZ5FsInLproMin',
 'Str2WasserZ5LsInLproMin',
 'Str2WasserZ3aFsInLproMin',
 'Str2WasserZ3aLsInLproMin',
'Str1WasserZ1DiefflenInLproMin',
 'Str1WasserZ1DillingenInLproMin'} 
  
water_keys_str1 = [ele for ele in discr if ele not in unwanted] 

In [ ]:
water_keys_str1 = sorted(water_keys_str1)
water_keys_str1

In [ ]:
interpol_func = {}
for key in water_keys_str1:
    name = key.replace('Str1Wasser','').replace('InLproMin','').replace('Z','z')
    minutes = data[key]['TIME'].to_numpy() / 60
    value = data[key][key].to_numpy()
    interpol_func[name] = interp1d(minutes,value, kind='linear')
    

In [ ]:
import scipy
def integrate(X,z,side):
        Y = [interpol_func[z+side](x) for x in X]
        return scipy.integrate.trapz(Y,X, dx=1)

def trapz_integration(row):
    begin_m = row[z + '_begin_time'] / 60
    end_m = row[z + '_end_time'] / 60
    try:
        row['water_' + z + '_' + side + '_inL'] = integrate([begin_m, end_m],z,side)
        return row
    except ValueError:
        return np.nan
                  

In [ ]:
zones_water = ['z1','z2a','z2b','z3a','z3b','z4','z5']
sides = ['Fs','Ls']
for side in tqdm(sides):
    for z in zones_water:
        str1_df = str1_df.apply(trapz_integration,axis=1)

In [ ]:
str1_df.head()

### Add the mean of the temperatur at the pyrometer

### Mean with Integral

In [ ]:
temp_keys = ['Str1TempMittelLsInC', 'Str1TempMittelFsInC']

from scipy.interpolate import interp1d
interpol_function_temp = {}
for key in temp_keys:
    minutes = data[key]['TIME'].to_numpy() / 60
    value = data[key][key].to_numpy()
    interpol_function_temp[key] = interp1d(minutes,value, kind='linear')

import scipy
def integrate_mean(X,key):
        Y = [interpol_function_temp[key](x) for x in X]
        return scipy.integrate.trapz(Y,X, dx=1)

def temp_mean(row):
    begin_t = row['zpy_begin_time'] / 60
    end_t= row['zpy_end_time'] / 60
    try:
        row[key + '_mean'] = (1 / (end_t - begin_t)) * integrate_mean([begin_t, end_t],key)
        return row
    except ValueError:
        return np.nan


In [ ]:
for key in tqdm(temp_keys):
    str1_df = str1_df.apply(temp_mean,axis=1)

### Merge str1_df with df_schmelzen to get other attributes

In [ ]:
str1_df = str1_df.rename(columns={'ChargenNr_begin': 'ChargenNr'})

In [ ]:
str1_df = pd.merge(str1_df,df_schmelzen,on='ChargenNr',how='inner')                                        

In [ ]:
str1_df.head()

### Sequence id

In [ ]:
str1_df['strang_nr'] = 1

In [ ]:
str1_df.head()

In [ ]:
def seq_and_str_nr(row):
    row['seq_id'] = str(row['SequenzNr']) + '_str_' + str(row['strang_nr'])
    return row

str1_df = str1_df.apply(seq_and_str_nr, axis=1)

In [ ]:
str1_df.head()

In [ ]:
str1_df_v2 = str1_df.drop(['Str2Format','Str2SollGiessGeschwInMproMin'],axis=1).rename(
    columns={'Str1Format': 'Format', 'Str1SollGiessGeschwInMproMin': 'SollGiessGeschwInMproMin' })

str1_df_v2.head()

### Save dataframe

In [ ]:
str1_df_v2.to_pickle("../final_data/df_str1_transformed.pkl")